# MNIST: Framework Implementation and Adversarial Robustness Analysis

## Imports and MNIST loading

In [1]:
# Imports all the module paths
import sys
sys.path.append("../../")

# Requirements for running everything
import torch

# File containing all the required training methods
import defences.mnist_library as mnist_library

# For testing
import utils.clean_test as clean_test

# Contains the data loadders
import utils.dataloaders as dataloaders

# For printing outcomes
# import utils.printing as printing

# Example printing, but I removed it to simplify results
# for epsilon in epsilons:
#     printing.print_attack(
#         model,
#         testSetLoader,
#         "FGSM",
#         attacks["FGSM"],
#         epsilon=epsilon,
#     )

Notebook will use PyTorch Device: CUDA
Notebook will use PyTorch Device: CUDA
Notebook will use PyTorch Device: CUDA
Notebook will use PyTorch Device: CUDA
Notebook will use PyTorch Device: CUDA


## Load the dataset

In [2]:
DATA_ROOT = "../../datasets/"

trainSetLoader, _, testSetLoader = dataloaders.get_MNIST_data_loaders(
    DATA_ROOT,
    trainSetSize=50000,
    validationSetSize=0,
    batchSize=128,
)

## Save path

In [3]:
SAVE_LOAD_ROOT = "../../data/mnist"

## Load the Attacks For Training

In [4]:
# A possible attacks array (for nice printing):
# Some attacks use a helper library
import torchattacks

import attacks.fgsm as fgsm
import attacks.ifgsm as ifgsm
import attacks.pgd as pgd
import utils.attacking as attacking

attacks = {}

attacks["FGSM"] = fgsm.fgsm_attack
attacks["I-FGSM"] = ifgsm.ifgsm_attack
attacks["PGD"] = pgd.pgd_attack

## Training Phase: Jacobian Regularizared PG+$CW_2$ Adversarially Trained Model (i.e. 2-Attack Adversarial Training)

In [5]:
# Pass none since if there is no attack function specified, will use CW2 instead
framework_model = mnist_library.framework_training(
    trainSetLoader,
    attack_function1=attacks["PGD"],
    attack_function2=None,
    load_if_available=True,
    load_path=SAVE_LOAD_ROOT + "/mnist_framework",
    epsilon=0.35,
    alpha=(2 / 255),
    iterations=7,
    steps=500,
    c=1,
)

Found already trained model...
... loaded!


In [6]:
clean_test.test_trained_model(framework_model, testSetLoader)

Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 98.24%


In [7]:
# Save the model
torch.save(framework_model, SAVE_LOAD_ROOT + "/mnist_framework")

### Evaluation

#### FGSM

In [8]:
# Several values to use for the epsilons
epsilons = [0, 0.01, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

# Run test for each epsilon
for epsilon in epsilons:
    attacking.attack_model(
        framework_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

Testing the model under FGSM Attack using epsilon = 0, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 98.24%
------------------------------------

Testing the model under FGSM Attack using epsilon = 0.01, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 97.55%
------------------------------------

Testing the model under FGSM Attack using epsilon = 0.05, alpha = None...


...

... done! Accuracy: 4.65%
------------------------------------

Testing the model under FGSM Attack using epsilon = 0.75, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 3.21%
------------------------------------

Testing the model under FGSM Attack using epsilon = 1, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 3.19%
------------------------------------



#### PGD

In [9]:
# Several values to use for the epsilons
epsilons = [0, 0.01, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

for epsilon in epsilons:
    attacking.attack_model(
        framework_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

Testing the model under PGD Attack using epsilon = 0, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 98.24%
------------------------------------

Testing the model under PGD Attack using epsilon = 0.01, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 97.51%
------------------------------------

Testing the model under PGD Attack using epsilon = 0.05, alpha = 0.00784313725490196...


...

... done! Accuracy: 92.8%
------------------------------------

Testing the model under PGD Attack using epsilon = 0.75, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 79.4%
------------------------------------

Testing the model under PGD Attack using epsilon = 1, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 36.77%
------------------------------------



#### $CW_2$

In [10]:
step_nums = [50, 100, 500]
cs = [1, 3, 5]

for c in cs:
    for step_num in step_nums:
        cw_attack = torchattacks.CW(framework_model, c=c, steps=step_num)
        attacks["CW"] = cw_attack

        attacking.attack_model(
        framework_model,
        testSetLoader,
        "CW",
        attacks["CW"],
        library=True,
        )

Testing the model under CW Attack...


CW Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 96.47%
------------------------------------

Testing the model under CW Attack...


CW Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 95.42%
------------------------------------

Testing the model under CW Attack...


...

... done! Accuracy: 95.56%
------------------------------------

Testing the model under CW Attack...


CW Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 90.83%
------------------------------------

Testing the model under CW Attack...


CW Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 43.63%
------------------------------------



## Detection Phase: PCA-based Detection

In [11]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.decomposition import PCA
from torch.autograd import Variable
from tqdm.notebook import tnrange, tqdm

# Define the `device` PyTorch will be running on, please hope it is CUDA
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Notebook will use PyTorch Device: " + device.upper())

# Copy the MNIST data and then fit using PCA
# First convert to numpy arrays (and make it float)
numpyTrainingData = trainSetLoader.dataset.data.numpy().astype("float32")
# Note you also need to reshape the input data for your sanity
reshapedNumpyTrainingData = numpyTrainingData.reshape((len(numpyTrainingData), 28 * 28))

# Then perform PCA on training data to get principal components
# Note it should reflect dimension of image, i.e. 28 * 28
pca = PCA(n_components=28 * 28).fit(reshapedNumpyTrainingData)

# For testing
loss_function = torch.nn.CrossEntropyLoss()

Notebook will use PyTorch Device: CUDA


#### Benign Data

In [12]:
# Now on clean data check if there are any adversarial samples
numpyTestData = testSetLoader.dataset.data.numpy().astype("float32")
reshapedNumpyTestData = numpyTestData.reshape((len(numpyTestData), 28 * 28))

# Original predictions on data
testTensor = torch.from_numpy(
    np.reshape(numpyTestData, (len(numpyTestData), 1, 28, 28))
).to(device)
logits = framework_model(testTensor).detach().cpu().numpy()
predictions_base = np.array([np.argmax(logits[i]) for i in range(len(numpyTestData))])

# Transform clean data along principal components
transformedTestData = pca.transform(reshapedNumpyTestData)

# Decides how many of the least significant coefficients (of components) to perturb
num_components = 200

# How many trials to run
num_trials = 25

# Track results
result = np.zeros(len(numpyTestData), dtype=int)

# Actual attempts
for trial in range(num_trials):
    random_noise = np.random.standard_normal(size=num_components)

    # Copy the data
    transformedTestDataNoisy = np.copy(transformedTestData)

    # Update the components with the right data
    for index in range(len(numpyTestData)):
        transformedTestDataNoisy[index][(28 * 28 - num_components) :] += (
            10 * random_noise
        )

    # Now calculate the inverse using PCA and the noise
    inverseTestDataNoisy = pca.inverse_transform(transformedTestDataNoisy)

    # Reshape into image
    testDataNoisy = np.reshape(inverseTestDataNoisy, (len(numpyTestData), 1, 28, 28))

    # Predict
    testTensor = torch.from_numpy(
        np.reshape(testDataNoisy, (len(testDataNoisy), 1, 28, 28))
    ).to(device)
    logits = framework_model(testTensor).detach().cpu().numpy()
    predictions_modified = np.array(
        [np.argmax(logits[i]) for i in range(len(testDataNoisy))]
    )

    check = np.not_equal(predictions_modified, predictions_base)
    result = np.logical_or(check, result)

# Printing
print(np.sum(result))

9


#### FGSM Data

In [13]:
# Now do the same on adversarial data check if there are any adversarial samples
# Use a pretty progress bar to show updates
data = []

for j, (images, labels) in enumerate(
    tqdm(testSetLoader, desc="Testing Progress", leave=False)
):
    # Cast to proper tensor
    images, labels = images.to(device), labels.to(device)

    # Perturb the images using the attack
    perturbed_images = fgsm.fgsm_attack(
        images,
        labels,
        framework_model,
        loss_function,
        epsilon=0.75,
        alpha=None,
        scale=True,
        iterations=None,
    )

    for perturbed_image in perturbed_images:
        data.append(perturbed_image.detach().cpu().numpy())

data = np.asarray(data)
numpyTestData = data.astype("float32")
reshapedNumpyTestData = numpyTestData.reshape((len(numpyTestData), 28 * 28))

# Original predictions on data
testTensor = torch.from_numpy(
    np.reshape(numpyTestData, (len(numpyTestData), 1, 28, 28))
).to(device)
logits = framework_model(testTensor).detach().cpu().numpy()
predictions_base = np.array([np.argmax(logits[i]) for i in range(len(numpyTestData))])

# Transform clean data along principal components
transformedTestData = pca.transform(reshapedNumpyTestData)

# Decides how many of the least significant coefficients (of components) to perturb
num_components = 200

# How many trials to run
num_trials = 25

# Track results
result = np.zeros(len(numpyTestData), dtype=int)

# Actual attempts
for trial in range(num_trials):
    random_noise = np.random.standard_normal(size=num_components)

    # Copy the data
    transformedTestDataNoisy = np.copy(transformedTestData)

    # Update the components with the right data
    for index in range(len(numpyTestData)):
        transformedTestDataNoisy[index][(28 * 28 - num_components) :] += (
            10 * random_noise
        )

    # Now calculate the inverse using PCA and the noise
    inverseTestDataNoisy = pca.inverse_transform(transformedTestDataNoisy)

    # Reshape into image
    testDataNoisy = np.reshape(inverseTestDataNoisy, (len(numpyTestData), 1, 28, 28))

    # Predict
    testTensor = torch.from_numpy(
        np.reshape(testDataNoisy, (len(testDataNoisy), 1, 28, 28))
    ).to(device)
    logits = logits = framework_model(testTensor).detach().cpu().numpy()
    predictions_modified = np.array(
        [np.argmax(logits[i]) for i in range(len(testDataNoisy))]
    )

    check = np.not_equal(predictions_modified, predictions_base)
    result = np.logical_or(check, result)

# Printing
print(np.sum(result))

Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

9923


#### PGD Data

In [14]:
# Now do the same on adversarial data check if there are any adversarial samples
# Use a pretty progress bar to show updates
data = []

for j, (images, labels) in enumerate(
    tqdm(testSetLoader, desc="Testing Progress", leave=False)
):
    # Cast to proper tensor
    images, labels = images.to(device), labels.to(device)

    # Perturb the images using the attack
    perturbed_images = pgd.pgd_attack(
        images,
        labels,
        framework_model,
        loss_function,
        epsilon=0.0001,
        alpha=(2 / 255),
        iterations=20,
        scale=True,
    )

    for perturbed_image in perturbed_images:
        data.append(perturbed_image.detach().cpu().numpy())

data = np.asarray(data)
numpyTestData = data.astype("float32")
reshapedNumpyTestData = numpyTestData.reshape((len(numpyTestData), 28 * 28))

# Original predictions on data
testTensor = torch.from_numpy(
    np.reshape(numpyTestData, (len(numpyTestData), 1, 28, 28))
).to(device)
logits = framework_model(testTensor).detach().cpu().numpy()
predictions_base = np.array([np.argmax(logits[i]) for i in range(len(numpyTestData))])

# Transform clean data along principal components
transformedTestData = pca.transform(reshapedNumpyTestData)

# Decides how many of the least significant coefficients (of components) to perturb
num_components = 200

# How many trials to run
num_trials = 25

# Track results
result = np.zeros(len(numpyTestData), dtype=int)

# Actual attempts
for trial in range(num_trials):
    random_noise = np.random.standard_normal(size=num_components)

    # Copy the data
    transformedTestDataNoisy = np.copy(transformedTestData)

    # Update the components with the right data
    for index in range(len(numpyTestData)):
        transformedTestDataNoisy[index][(28 * 28 - num_components) :] += (
            10 * random_noise
        )

    # Now calculate the inverse using PCA and the noise
    inverseTestDataNoisy = pca.inverse_transform(transformedTestDataNoisy)

    # Reshape into image
    testDataNoisy = np.reshape(inverseTestDataNoisy, (len(numpyTestData), 1, 28, 28))

    # Predict
    testTensor = torch.from_numpy(
        np.reshape(testDataNoisy, (len(testDataNoisy), 1, 28, 28))
    ).to(device)
    logits = logits = framework_model(testTensor).detach().cpu().numpy()
    predictions_modified = np.array(
        [np.argmax(logits[i]) for i in range(len(testDataNoisy))]
    )

    check = np.not_equal(predictions_modified, predictions_base)
    result = np.logical_or(check, result)

# Printing
print(np.sum(result))

Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

9969


In [15]:
# Now do the same on adversarial data check if there are any adversarial samples
# Use a pretty progress bar to show updates
data = []

for j, (images, labels) in enumerate(
    tqdm(testSetLoader, desc="Testing Progress", leave=False)
):
    # Cast to proper tensor
    images, labels = images.to(device), labels.to(device)

    # Perturb the images using the attack
    perturbed_images = pgd.pgd_attack(
        images,
        labels,
        framework_model,
        loss_function,
        epsilon=0.01,
        alpha=(2 / 255),
        iterations=200,
        scale=True,
    )

    for perturbed_image in perturbed_images:
        data.append(perturbed_image.detach().cpu().numpy())

data = np.asarray(data)
numpyTestData = data.astype("float32")
reshapedNumpyTestData = numpyTestData.reshape((len(numpyTestData), 28 * 28))

# Original predictions on data
testTensor = torch.from_numpy(
    np.reshape(numpyTestData, (len(numpyTestData), 1, 28, 28))
).to(device)
logits = framework_model(testTensor).detach().cpu().numpy()
predictions_base = np.array([np.argmax(logits[i]) for i in range(len(numpyTestData))])

# Transform clean data along principal components
transformedTestData = pca.transform(reshapedNumpyTestData)

# Decides how many of the least significant coefficients (of components) to perturb
num_components = 200

# How many trials to run
num_trials = 25

# Track results
result = np.zeros(len(numpyTestData), dtype=int)

# Actual attempts
for trial in range(num_trials):
    random_noise = np.random.standard_normal(size=num_components)

    # Copy the data
    transformedTestDataNoisy = np.copy(transformedTestData)

    # Update the components with the right data
    for index in range(len(numpyTestData)):
        transformedTestDataNoisy[index][(28 * 28 - num_components) :] += (
            10 * random_noise
        )

    # Now calculate the inverse using PCA and the noise
    inverseTestDataNoisy = pca.inverse_transform(transformedTestDataNoisy)

    # Reshape into image
    testDataNoisy = np.reshape(inverseTestDataNoisy, (len(numpyTestData), 1, 28, 28))

    # Predict
    testTensor = torch.from_numpy(
        np.reshape(testDataNoisy, (len(testDataNoisy), 1, 28, 28))
    ).to(device)
    logits = logits = framework_model(testTensor).detach().cpu().numpy()
    predictions_modified = np.array(
        [np.argmax(logits[i]) for i in range(len(testDataNoisy))]
    )

    check = np.not_equal(predictions_modified, predictions_base)
    result = np.logical_or(check, result)

# Printing
print(np.sum(result))

Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

9541


In [16]:
# Now do the same on adversarial data check if there are any adversarial samples
# Use a pretty progress bar to show updates
data = []

for j, (images, labels) in enumerate(
    tqdm(testSetLoader, desc="Testing Progress", leave=False)
):
    # Cast to proper tensor
    images, labels = images.to(device), labels.to(device)

    # Perturb the images using the attack
    perturbed_images = pgd.pgd_attack(
        images,
        labels,
        framework_model,
        loss_function,
        epsilon=0.85,
        alpha=(2 / 255),
        iterations=200,
        scale=True,
    )

    for perturbed_image in perturbed_images:
        data.append(perturbed_image.detach().cpu().numpy())

data = np.asarray(data)
numpyTestData = data.astype("float32")
reshapedNumpyTestData = numpyTestData.reshape((len(numpyTestData), 28 * 28))

# Original predictions on data
testTensor = torch.from_numpy(
    np.reshape(numpyTestData, (len(numpyTestData), 1, 28, 28))
).to(device)
logits = framework_model(testTensor).detach().cpu().numpy()
predictions_base = np.array([np.argmax(logits[i]) for i in range(len(numpyTestData))])

# Transform clean data along principal components
transformedTestData = pca.transform(reshapedNumpyTestData)

# Decides how many of the least significant coefficients (of components) to perturb
num_components = 300

# How many trials to run
num_trials = 25

# Track results
result = np.zeros(len(numpyTestData), dtype=int)

# Actual attempts
for trial in range(num_trials):
    random_noise = np.random.standard_normal(size=num_components)

    # Copy the data
    transformedTestDataNoisy = np.copy(transformedTestData)

    # Update the components with the right data
    for index in range(len(numpyTestData)):
        transformedTestDataNoisy[index][(28 * 28 - num_components) :] += (
            10 * random_noise
        )

    # Now calculate the inverse using PCA and the noise
    inverseTestDataNoisy = pca.inverse_transform(transformedTestDataNoisy)

    # Reshape into image
    testDataNoisy = np.reshape(inverseTestDataNoisy, (len(numpyTestData), 1, 28, 28))

    # Predict
    testTensor = torch.from_numpy(
        np.reshape(testDataNoisy, (len(testDataNoisy), 1, 28, 28))
    ).to(device)
    logits = logits = framework_model(testTensor).detach().cpu().numpy()
    predictions_modified = np.array(
        [np.argmax(logits[i]) for i in range(len(testDataNoisy))]
    )

    check = np.not_equal(predictions_modified, predictions_base)
    result = np.logical_or(check, result)

# Printing
print(np.sum(result))

Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

9998


#### $CW_{2}$ Data

In [17]:
cw_attack = torchattacks.CW(framework_model, c=1, steps=500)

# Now do the same on adversarial data check if there are any adversarial samples
# Use a pretty progress bar to show updates
data = []

for j, (images, labels) in enumerate(
    tqdm(testSetLoader, desc="Testing Progress", leave=False)
):
    # Cast to proper tensor
    images, labels = images.to(device), labels.to(device)

    # Perturb the images using the attack
    perturbed_images = cw_attack(
        images,
        labels,
    )

    for perturbed_image in perturbed_images:
        data.append(perturbed_image.detach().cpu().numpy())

data = np.asarray(data)
numpyTestData = data.astype("float32")
reshapedNumpyTestData = numpyTestData.reshape((len(numpyTestData), 28 * 28))

# Original predictions on data
testTensor = torch.from_numpy(
    np.reshape(numpyTestData, (len(numpyTestData), 1, 28, 28))
).to(device)
logits = framework_model(testTensor).detach().cpu().numpy()
predictions_base = np.array([np.argmax(logits[i]) for i in range(len(numpyTestData))])

# Transform clean data along principal components
transformedTestData = pca.transform(reshapedNumpyTestData)

# Decides how many of the least significant coefficients (of components) to perturb
num_components = 200

# How many trials to run
num_trials = 25

# Track results
result = np.zeros(len(numpyTestData), dtype=int)

# Actual attempts
for trial in range(num_trials):
    random_noise = np.random.standard_normal(size=num_components)

    # Copy the data
    transformedTestDataNoisy = np.copy(transformedTestData)

    # Update the components with the right data
    for index in range(len(numpyTestData)):
        transformedTestDataNoisy[index][(28 * 28 - num_components) :] += (
            10 * random_noise
        )

    # Now calculate the inverse using PCA and the noise
    inverseTestDataNoisy = pca.inverse_transform(transformedTestDataNoisy)

    # Reshape into image
    testDataNoisy = np.reshape(inverseTestDataNoisy, (len(numpyTestData), 1, 28, 28))

    # Predict
    testTensor = torch.from_numpy(
        np.reshape(testDataNoisy, (len(testDataNoisy), 1, 28, 28))
    ).to(device)
    logits = logits = framework_model(testTensor).detach().cpu().numpy()
    predictions_modified = np.array(
        [np.argmax(logits[i]) for i in range(len(testDataNoisy))]
    )

    check = np.not_equal(predictions_modified, predictions_base)
    result = np.logical_or(check, result)

# Printing
print(np.sum(result))

Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

9456


In [18]:
cw_attack = torchattacks.CW(framework_model, c=3, steps=500)

# Now do the same on adversarial data check if there are any adversarial samples
# Use a pretty progress bar to show updates
data = []

for j, (images, labels) in enumerate(
    tqdm(testSetLoader, desc="Testing Progress", leave=False)
):
    # Cast to proper tensor
    images, labels = images.to(device), labels.to(device)

    # Perturb the images using the attack
    perturbed_images = cw_attack(
        images,
        labels,
    )

    for perturbed_image in perturbed_images:
        data.append(perturbed_image.detach().cpu().numpy())

data = np.asarray(data)
numpyTestData = data.astype("float32")
reshapedNumpyTestData = numpyTestData.reshape((len(numpyTestData), 28 * 28))

# Original predictions on data
testTensor = torch.from_numpy(
    np.reshape(numpyTestData, (len(numpyTestData), 1, 28, 28))
).to(device)
logits = framework_model(testTensor).detach().cpu().numpy()
predictions_base = np.array([np.argmax(logits[i]) for i in range(len(numpyTestData))])

# Transform clean data along principal components
transformedTestData = pca.transform(reshapedNumpyTestData)

# Decides how many of the least significant coefficients (of components) to perturb
num_components = 200

# How many trials to run
num_trials = 25

# Track results
result = np.zeros(len(numpyTestData), dtype=int)

# Actual attempts
for trial in range(num_trials):
    random_noise = np.random.standard_normal(size=num_components)

    # Copy the data
    transformedTestDataNoisy = np.copy(transformedTestData)

    # Update the components with the right data
    for index in range(len(numpyTestData)):
        transformedTestDataNoisy[index][(28 * 28 - num_components) :] += (
            10 * random_noise
        )

    # Now calculate the inverse using PCA and the noise
    inverseTestDataNoisy = pca.inverse_transform(transformedTestDataNoisy)

    # Reshape into image
    testDataNoisy = np.reshape(inverseTestDataNoisy, (len(numpyTestData), 1, 28, 28))

    # Predict
    testTensor = torch.from_numpy(
        np.reshape(testDataNoisy, (len(testDataNoisy), 1, 28, 28))
    ).to(device)
    logits = logits = framework_model(testTensor).detach().cpu().numpy()
    predictions_modified = np.array(
        [np.argmax(logits[i]) for i in range(len(testDataNoisy))]
    )

    check = np.not_equal(predictions_modified, predictions_base)
    result = np.logical_or(check, result)

# Printing
print(np.sum(result))

Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

9575


## Detection Phase: Mahalanobis-based Detection

Please see in the **defences/mahalanobis_detector/** folder how to run the Mahalanobis-based Detection component, as it has a separate procedure.